In [1]:
!pip install deap

     ------------------------------------ 109.9/109.9 kB 491.0 kB/s eta 0:00:00


In [1]:
import random
from deap import base, creator, tools, algorithms

# Placeholder evaluation function (replace this with your actual implementation)
def evaluate(individual):
    # Your actual evaluation logic here (e.g., train a neural network and compute fitness)
    # This example returns a random fitness value for demonstration purposes
    return random.random(),  # Note: Fitness values must be returned as a tuple

# Genetic algorithm parameters
POPULATION_SIZE = 10
GENERATIONS = 5

# Create types for fitness and individuals
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize fitness
creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialize toolbox
toolbox = base.Toolbox()

# Define attributes and individuals
toolbox.register("attr_neurons", random.randint, 10, 100)  # Number of neurons
toolbox.register("attr_layers", random.randint, 2, 5)      # Number of layers
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_neurons, toolbox.attr_layers), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=10, up=100, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create initial population
population = toolbox.population(n=POPULATION_SIZE)

# Run the genetic algorithm
for gen in range(GENERATIONS):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)

    # Evaluate the offspring
    fitnesses = map(toolbox.evaluate, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

    # Select the next generation
    population = toolbox.select(offspring, k=len(population))

# Get the best individual and parameters
best_individual = tools.selBest(population, k=1)[0]
best_params = best_individual

print("Best Parameters (Neurons per layer, Number of layers):", best_params)



Best Parameters (Neurons per layer, Number of layers): [18, 71]


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Load your actual data (replace this with your data loading logic)
def load_data():
    X = np.random.rand(100, 5)  # Example: Random feature matrix
    y = np.random.rand(100)      # Example: Random target vector
    return X, y

# Evaluation function
def evaluate(individual):
    X, y = load_data()  # Load your data

    # Extract parameters from the individual
    hidden_layer_sizes = (individual[0],) * individual[1]  # Example: (10, 10, 10) for 3 layers with 10 neurons each
    learning_rate_init = individual[2]                     # Example: Learning rate
    momentum = individual[3]                               # Example: Momentum

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train neural network
    nn = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes,
                      learning_rate_init=learning_rate_init,
                      momentum=momentum,
                      max_iter=500)  # Adjust max_iter as needed
    nn.fit(X_train, y_train)

    # Evaluate performance on validation set (test set)
    y_pred = nn.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Return fitness value (minimize mean squared error)
    return mse,

# Example usage of the evaluate function
# Replace this with your actual implementation
# Example individual representing (number of neurons, number of layers, learning rate, momentum)
individual = [10, 3, 0.01, 0.9]  # Example individual
fitness_value = evaluate(individual)
print("Fitness Value:", fitness_value)


Fitness Value: (0.14053872812496473,)


In [14]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Load and preprocess the dataset (example with the diabetes dataset)
def load_data():
    diabetes = load_diabetes()
    X_train, X_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Evaluation function
def evaluate(individual):
    # Extract parameters
    hidden_layer_sizes = tuple(individual[0:2])
    learning_rate_init = individual[2]
    
    # Load and preprocess data
    X_train, X_test, y_train, y_test = load_data()
    
    # Initialize and train neural network
    nn = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes,
                      learning_rate_init=learning_rate_init,
                      max_iter=1000)  # Adjust max_iter as needed
    nn.fit(X_train, y_train)
    
    # Evaluate performance on validation set
    y_pred = nn.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # Return fitness value (minimize MSE)
    return mse,

# Example usage:
individual = [10, 20, 0.001]  # Example individual (neurons in hidden layers, learning rate)
fitness_value = evaluate(individual)
print("Fitness value:", fitness_value[0])


Fitness value: 3024.391025999067


c:\Users\Pratham\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
